Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 1: Set Covering

First lab + peer review. List this activity in your final report, it will be part of your exam.

## Task

Given a number $N$ and some lists of integers $P = (L_0, L_1, L_2, ..., L_n)$, 
determine is possible $S = (L_{s_0}, L_{s_1}, L_{s_2}, ..., L_{s_n})$
such that each number between $0$ and $N-1$ appears in at least one list

$$\forall n \in [0, N-1] \ \exists i : n \in L_{s_i}$$

and that the total numbers of elements in all $L_{s_i}$ is minimum. ($\min \sum_i^{n}len(L_{s_i})$)

## Instructions

* Create the directory `lab1` inside the course repo (the one you registered with Andrea)
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)
* Use `problem` to generate the problems with different $N$
* In the `README.md`, report the the total numbers of elements in $L_{s_i}$ for problem with $N \in [5, 10, 20, 100, 500, 1000]$ and the total number on $nodes$ visited during the search. Use `seed=42`.
* Use `GitHub Issues` to peer review others' lab

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

**Deadline**

* Sunday, October 16th 23:59:59 for the working solution
* Sunday, October 23rd 23:59:59 for the peer reviews

# Code

In [8]:
import random
import numpy as np
import logging
import pprint
from utils import search, State
logging.basicConfig(format="%(asctime)s %(levelname)s: %(message)s")

In [9]:
DEBUG = False
logging.getLogger().setLevel(logging.DEBUG if DEBUG else logging.INFO)

In [10]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [11]:
SEED = 42

In [12]:
INITIAL_STATE = State(())

In [13]:
def possible_actions(state: State):
    for p in P:
        data = np.zeros(N)
        data[p] = 1
        if len(state.data) == 0 or not (np.logical_and(state.data[0], data) == data).all():
            yield np.array(p)

def result(state: State, action):
    action_bit = np.zeros(N, dtype=bool)
    action_bit[action] = True
    if len(state.data) == 0:
        return State((action_bit, action))
    result_history = np.logical_or(state.data[0], action_bit)
    del action_bit
    return State((result_history, action))

def goal_test(state: State):
    if len(state.data) == 0:
        return False    
    return state.data[0].all()

In [14]:
parent_state = dict()
state_cost = dict()


def h(state):
    return N - state.data[0].sum()

for N in [5, 10, 20]:
    P = problem(N, SEED)
    logging.info(f"P: {P}")
    U = np.arange(N)
    UxS = np.zeros((N, len(P)), dtype=np.uint8)
    for i, p in enumerate(P):
        UxS[p, i] = 1

    if not UxS.any(axis=1).all():
        logging.info("Impossible")
        exit
    
    final = search(
        INITIAL_STATE,
        goal_test=goal_test,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: state_cost[s] + h(s),
        unit_cost=lambda a: len(a),
        possible_actions=possible_actions,
        result=result
    )

    final_prettier = [list(final[i][1]) for i in range(1, len(final))]
    logging.info(f"N: {N}, W: {sum(map(lambda x: len(x), final_prettier))}, Solution: {final_prettier}")

2022-10-24 23:10:24,538 INFO: P: [[0], [1], [0], [4], [0], [1], [4], [4], [4], [1, 3], [0, 1], [2], [1], [0], [0, 2], [2, 4], [3], [3], [4], [2, 4], [0], [1], [0, 1], [3], [2, 3]]
2022-10-24 23:10:24,544 INFO: Found a solution in 4 steps; visited 3 nodes
2022-10-24 23:10:24,545 INFO: N: 5, W: 5
2022-10-24 23:10:24,545 INFO: P: [[0, 4], [1, 2, 3], [9, 6], [0, 1], [8, 9, 3], [8, 3], [0, 3, 4, 7, 9], [4, 5, 6], [1, 3, 5], [1, 6], [0, 9, 4, 5], [8, 1, 6], [9, 3, 5], [0, 3], [1, 3, 6], [2, 5, 7], [1, 3, 4, 9], [8, 2, 3], [3, 4, 5, 6, 8], [0, 3], [1, 3, 4, 6], [3, 6, 7], [2, 3, 4], [9, 6], [8, 2, 3, 7], [0, 1], [9, 2, 6], [6], [8, 0, 4, 1], [1, 4, 5, 6], [0, 4, 7], [8, 1, 4], [2, 5], [9, 5], [0, 1, 3, 4, 5], [9, 3], [1, 7], [8, 2], [8, 2, 7], [8, 9, 3, 6], [4, 5, 6], [8, 1, 3, 7], [0, 5], [0, 9, 3], [0, 3], [0, 5], [8, 3], [8, 2, 3, 7], [1, 3, 6, 7], [5, 6]]
2022-10-24 23:10:24,557 INFO: Found a solution in 4 steps; visited 3 nodes
2022-10-24 23:10:24,558 INFO: N: 10, W: 10
2022-10-24 23:10: